In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_filer_date = dbutils.widgets.get("p_file_date")


#### Produce Driver Standings

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Find the Race  year for which  the data is reprocessed 

In [0]:
# race_result_list = spark.read.parquet(f"{presentation_folder_path}/race_results") \
#     .filter(f"file_date = '{v_filer_date}'") \
#     .select("race_year") \
#     .distinct() \
#     .collect()   

race_result_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(f"file_date = '{v_filer_date}'") 

In [0]:
# race_result_list
race_year_list = df_column_to_list(race_result_df,"race_year")

In [0]:
# race_year_list = []
# for race_year in race_result_list:
#     race_year_list.append(race_year.race_year)
# print(race_year_list)


In [0]:
from pyspark.sql.functions import col

In [0]:
race_result_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(col("race_year").isin(race_year_list))


In [0]:
# display(race_result_df)

In [0]:
from pyspark.sql.functions import sum, count, when, col

driver_standings_df = race_result_df \
    .groupBy("race_year","driver_name","driver_nationality") \
    .agg(sum("points").alias("total_points"),count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(driver_standings_df.filter("race_year == 2020"))

race_year,driver_name,driver_nationality,total_points,wins


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc

driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"),desc("wins"))

final_df = driver_standings_df.withColumn("rank",rank().over(driver_rank_spec))

In [0]:
display(final_df.filter("race_year == 2020"))

race_year,driver_name,driver_nationality,total_points,wins,rank


In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/driver_standings")

# final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.driver_standings")

# overwrite_function(final_df, 'f1_presentation', 'driver_standings', 'race_year')


In [0]:
merge_condition = "tgt.driver_name = src.driver_name and tgt.race_year = src.race_year"
merge_delta_data(final_df, "f1_presentation", "driver_standings", "race_year",processed_folder_path, merge_condition)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3393798542034738>, line 2
      1 merge_condition = "tgt.driver_name = src.driver_name and tgt.race_year = src.race_year"
----> 2 merge_delta_data(final_df, "f1_presentation", "driver_standings", "race_year",processed_folder_path, merge_condition)

File <command-3651007263442800>, line 6, in merge_delta_data(input_df, db_name, table_name, partition_column, folder_path, merge_condition)
      4 def merge_delta_data(input_df, db_name,table_name,partition_column,folder_path,merge_condition):
      5     if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
----> 6       deltaTable = DeltaTable.forPath(spark,f"{folder_path}/{table_name}")
      7       deltaTable.alias('tgt').merge(
      8         input_df.alias("src"),
      9         merge_condition) \
     10         .whenMatchedUpdateAll() \
     11 

In [0]:
%sql

select * from f1_presentation.driver_standings where race_year = 2021;

In [0]:
%sql
select race_year,count(1) from f1_presentation.driver_standings
group by race_year
order by race_year desc;

In [0]:
%sql
-- drop table  f1_presentation.driver_standings;